## 라이브러리 

In [1]:
import matplotlib.pyplot as plt
plt.rc('font',family='Malgun Gothic')
import tensorflow as tf
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import matplotlib as mpl
from IPython.core.display import display, HTML
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, LSTM, Dropout,GRU
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from statsmodels.tsa.seasonal import seasonal_decompose, STL
import urllib.request
import json
import datetime
import os
from datetime import datetime
import warnings

warnings.filterwarnings('ignore')

# 2016.1.1 ~ 2020.9.28 농산물 Train 데이터 

In [2]:
# train data set 불러오기

train_df = pd.read_csv('nongsan_data/public_data/train.csv', encoding='utf-8')
train_df['date'] = pd.to_datetime(train_df['date'])
train_df.index = train_df['date']
train_df = train_df.drop('date',axis=1)
train_df.index.freq = 'D'
train_df = train_df.interpolate(method='linear').fillna(0)
train_df = train_df.reset_index()
train_df = pd.concat([train_df,pd.get_dummies(train_df['요일'])],axis=1)
train_df = train_df.drop('요일',axis=1)
train_df.head()

,date,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),건고추_거래량(kg),건고추_가격(원/kg),마늘_거래량(kg),...,캠벨얼리_가격(원/kg),샤인마스캇_거래량(kg),샤인마스캇_가격(원/kg),금요일,목요일,수요일,월요일,일요일,토요일,화요일
0,2016-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1,0,0,0,0,0,0
1,2016-01-02,80860.0,329.0,80272.0,360.0,122787.5,1281.0,3.0,11000.0,15019.0,...,2014.0,0.0,0.0,0,0,0,0,0,1,0
2,2016-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,1,0,0
3,2016-01-04,1422742.5,478.0,1699653.7,382.0,2315079.0,1235.0,699.0,4464.0,141638.0,...,3885.0,0.0,0.0,0,0,0,1,0,0,0
4,2016-01-05,1167241.0,442.0,1423482.3,422.0,2092960.1,1213.0,1112.6,4342.0,126207.8,...,2853.0,0.0,0.0,0,0,0,0,0,0,1


# 2020.9.29 ~ 2020.11.04 농산물 Test 데이터 

In [3]:
# testdata set 불러오기

test_df = pd.read_csv('nongsan_data/public_data/test_files/test_2020-11-05.csv', encoding='utf-8')
test_df['date'] = pd.to_datetime(test_df['date'])
test_df.index = test_df['date']
test_df = test_df.drop('date',axis=1)
test_df.index.freq = 'D'
test_df = test_df.reset_index()
test_df = pd.concat([test_df,pd.get_dummies(test_df['요일'])],axis=1)
test_df = test_df.drop('요일',axis=1)
test_df.head()

,date,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),건고추_거래량(kg),건고추_가격(원/kg),마늘_거래량(kg),...,캠벨얼리_가격(원/kg),샤인마스캇_거래량(kg),샤인마스캇_가격(원/kg),금요일,목요일,수요일,월요일,일요일,토요일,화요일
0,2020-09-29,2064183.3,1787.0,2424383.2,1190.0,1209647.0,966.0,2842.8,25873.0,160209.8,...,3937.0,259765.0,11400.0,0,0,0,0,0,0,1
1,2020-09-30,259065.3,1551.0,1044460.0,984.0,136696.4,857.0,35.0,9929.0,48296.7,...,3885.0,121859.6,11649.0,0,0,1,0,0,0,0
2,2020-10-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,1,0,0,0,0,0
3,2020-10-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1,0,0,0,0,0,0
4,2020-10-03,0.0,0.0,3100.0,835.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,1,0


# 사용할 변수 지정

In [4]:
week_names = ['월요일','화요일','수요일','목요일','금요일','토요일','일요일']

## 농산물별 자동으로 변수 만들어주는 함수

In [5]:
def set_feature_per_item(item):
    price_feature = f'{item}_가격(원/kg)'
    quantity_feature = f'{item}_거래량(kg)'
    all_features = [price_feature,quantity_feature]
    all_features.extend(week_names)
    
    return price_feature,quantity_feature,set(all_features)

In [6]:
price_feature , quantity_feature , all_features = set_feature_per_item('배추')
price_feature , quantity_feature , all_features

('배추_가격(원/kg)',
 '배추_거래량(kg)',
 {'금요일',
  '목요일',
  '배추_가격(원/kg)',
  '배추_거래량(kg)',
  '수요일',
  '월요일',
  '일요일',
  '토요일',
  '화요일'})

## 모든 필요한 변수에 날짜 넣고, 필요없는 거래량 빼줌

In [7]:
all_features.add('date')
all_features = all_features - set(['배추_거래량(kg)'])

## 훈련용 데이터 7일 후의 배추가격을 target으로 지정

In [8]:
train_df = train_df[all_features]
train_df['7일 후'] = train_df[price_feature].shift(-7)
train_df['residual'] = 0
stl = STL(train_df[['date', price_feature]].set_index('date'), period=12)
res = stl.fit()
train_df['residual'] = res.resid.values
train_df.tail()

,월요일,일요일,토요일,금요일,date,수요일,배추_가격(원/kg),화요일,목요일,7일 후,residual
1728,0,0,0,0,2020-09-24,0,1839.0,0,1,NaN,-152.919637
1729,0,0,0,1,2020-09-25,0,1789.0,0,0,NaN,563.415158
1730,0,0,1,0,2020-09-26,0,1760.0,0,0,NaN,-359.340205
1731,0,1,0,0,2020-09-27,0,3066.0,0,0,NaN,204.297484
1732,1,0,0,0,2020-09-28,0,1867.0,0,0,NaN,-232.379198


## 테스트용 데이터 

In [9]:
cabbage_test_df = test_df[all_features]
cabbage_test_df['예측'] = 0 
cabbage_test_df.head()

,월요일,일요일,토요일,금요일,date,수요일,배추_가격(원/kg),화요일,목요일,예측
0,0,0,0,0,2020-09-29,0,1787.0,1,0,0
1,0,0,0,0,2020-09-30,1,1551.0,0,0,0
2,0,0,0,0,2020-10-01,0,0.0,0,1,0
3,0,0,0,1,2020-10-02,0,0.0,0,0,0
4,0,0,1,0,2020-10-03,0,0.0,0,0,0


# 모델 생성용 피쳐

In [10]:
model_features = all_features - set(['date'])
model_features.add('residual')
model_features

{'residual', '금요일', '목요일', '배추_가격(원/kg)', '수요일', '월요일', '일요일', '토요일', '화요일'}

In [11]:
my_minmax = MinMaxScaler()

In [12]:
train_df[list(model_features)] = my_minmax.fit_transform(train_df[list(model_features)])

# 학습데이터 NaN 부분제거

In [13]:
cabbage_train_df = train_df.iloc[:-7]

cabbage_train_df
cabbage_train_df.tail(10)

,월요일,일요일,토요일,금요일,date,수요일,배추_가격(원/kg),화요일,목요일,7일 후,residual
1716,0.0,0.0,1.0,0.0,2020-09-12,0.0,0.3496,0.0,0.0,1812.0,0.317480
1717,0.0,1.0,0.0,0.0,2020-09-13,0.0,0.0000,0.0,0.0,2925.0,0.012409
1718,1.0,0.0,0.0,0.0,2020-09-14,0.0,0.4084,0.0,0.0,1813.0,0.366676
1719,0.0,0.0,0.0,0.0,2020-09-15,0.0,0.4034,1.0,0.0,1838.0,0.209111
1720,0.0,0.0,0.0,0.0,2020-09-16,1.0,0.3878,0.0,0.0,1807.0,0.348318
1721,0.0,0.0,0.0,0.0,2020-09-17,0.0,0.3966,0.0,1.0,1839.0,0.260786
1722,0.0,0.0,0.0,1.0,2020-09-18,0.0,0.3678,0.0,0.0,1789.0,0.392283
1723,0.0,0.0,1.0,0.0,2020-09-19,0.0,0.3624,0.0,0.0,1760.0,0.206196
1724,0.0,1.0,0.0,0.0,2020-09-20,0.0,0.5850,0.0,0.0,3066.0,0.288483
1725,1.0,0.0,0.0,0.0,2020-09-21,0.0,0.3626,0.0,0.0,1867.0,0.211657


In [14]:
X = cabbage_train_df[model_features].values.reshape(-1,1,len(model_features))

In [15]:
y = cabbage_train_df['7일 후'].values.reshape(-1,1,1)

In [16]:
X.shape

(1726, 1, 9)

In [17]:
y.shape

(1726, 1, 1)

In [19]:
with tf.device('/CPU:0'):
    model = Sequential()
    model.add(layers.Activation('relu'))
    model.add(LSTM(100, input_shape=(21,len(model_features)), return_sequences=True))
    model.add(Dropout(0.1))
    model.add(layers.Dense(50))
    model.add(Dropout(0.1))
    model.add(Dropout(0.1))
    model.add(layers.Dense(1))
    model.compile(optimizer='adam', loss='mse')
    early_stopping = EarlyStopping(patience=50)
    model.fit(X, y, epochs=1000, batch_size=32, validation_split = 0.1, callbacks=[early_stopping], verbose=0)

In [ ]:
my_X = train_df.iloc[-7:][model_features].values.reshape(-1,1,len(model_features))

In [ ]:
predict_df = train_df.iloc[-7:][all_features]

In [ ]:
y_pred = model.predict(my_X)
y_pred = list(y_pred.reshape(-1,1).flatten())
predict_df['7일 후 예측'] = y_pred

In [ ]:
predict_df.iloc[2:6,-1] = 0

In [ ]:
predict_df['date'] = predict_df['date'].map(lambda x : x + relativedelta(days=7))

In [ ]:
test_df.iloc[:7][['date',price_feature]]

In [ ]:
predict_df[['date','7일 후 예측']]

In [ ]:
np.sqrt(mean_squared_error(test_df.iloc[:7][price_feature],predict_df['7일 후 예측']))

In [ ]:
mean_absolute_error(test_df.iloc[:7][price_feature],predict_df['7일 후 예측'])

In [ ]:
predict_df

In [ ]:
cabbage_train_df.head()

In [ ]:
cabbage_train_df['7일후 예측']= list(model.predict(X).reshape(-1,1).flatten())

In [ ]:
mean_absolute_error(cabbage_train_df['7일후 예측'],cabbage_train_df['7일 후'])

In [ ]:
np.sqrt(mean_squared_error(cabbage_train_df['7일후 예측'],cabbage_train_df['7일 후']))

In [ ]:
residual = cabbage_train_df['7일 후'] - cabbage_train_df['7일후 예측']